In [ ]:
# ✅ Install required libraries
!pip install -q transformers peft datasets

# ✅ Imports
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset

# ✅ Load WikiSQL dataset
dataset = load_dataset("wikisql")

# ✅ Use a small slice for testing (adjust sizes later!)
train_dataset = dataset["train"].select(range(5000))
eval_dataset = dataset["validation"].select(range(1000))

# ✅ Load CodeT5-small
model_name = "Salesforce/codet5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# ✅ Apply LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# ✅ Tokenization function (schema + question)
def tokenize_wikisql(batch):
    questions = batch["question"]
    sqls = [s["human_readable"] for s in batch["sql"]]
    table_headers = [" | ".join(t["header"]) for t in batch["table"]]
    inputs_text = [
        f"question: {q} table: {t}" for q, t in zip(questions, table_headers)
    ]
    model_inputs = tokenizer(
        inputs_text,
        padding="max_length",
        truncation=True,
        max_length=256
    )
    labels = tokenizer(
        sqls,
        padding="max_length",
        truncation=True,
        max_length=128
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# ✅ Tokenize datasets
tokenized_train = train_dataset.map(tokenize_wikisql, batched=True, remove_columns=train_dataset.column_names)
tokenized_eval = eval_dataset.map(tokenize_wikisql, batched=True, remove_columns=eval_dataset.column_names)

# ✅ Training arguments (safe for Colab Free)
training_args = Seq2SeqTrainingArguments(
    output_dir="./codet5-lora-sql",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-4,
    num_train_epochs=1,  # use 1 for testing
    logging_steps=10,
    save_steps=50,
    eval_strategy="steps",
    predict_with_generate=True,
    fp16=True  # saves VRAM on Colab
)

# ✅ Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ✅ Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# ✅ Train
trainer.train()


In [25]:
test_input = tokenizer(
    "question: How many students have a grade above 90? table: | name | grade |",
    return_tensors="pt"
).to("cuda")

output = model.generate(**test_input, max_length=64)
print(tokenizer.decode(output[0], skip_special_tokens=True))


SELECT COUNT name FROM table WHERE grade = 90


In [ ]:
model.save_pretrained("./SqlGPT")
tokenizer.save_pretrained("./SqlGPT")


In [ ]:
from huggingface_hub import login

login()  # paste your token

model.push_to_hub("Mahendra1742/SqlGPT")
tokenizer.push_to_hub("Mahendra1742/SqlGPT")


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Replace with your actual HF repo name
model_name = "Mahendra1742/SqlGPT"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Example input
question = "How many employees are in the Marketing department?"
table = "| department | employees |"

prompt = f"question: {question} table: {table}"

inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**inputs, max_length=128)

print("OUTPUT :- ")
print("   ")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))